Example of a RAG (Retrieval-Augmented Generation) system — often used in GenAI projects when you want an LLM like GPT or LLaMA to generate accurate, context-specific answers from your own documents or data.

# 🧠 What is RAG?
Retrieval-Augmented Generation (RAG) =
retrieval step + generation step

It allows an LLM to:

Retrieve relevant information from your data (documents, PDFs, DBs).

Generate an answer using that info + prompt.

# 📘 Example: "HR Document Assistant" (RAG)

# ✅ Goal:
“Build a chatbot that answers questions like ‘What is the company leave policy?’ based on your internal HR policy PDFs.”

# 🤖 RAG System: OpenAI vs Local Embeddings
This notebook compares two Retrieval-Augmented Generation (RAG) pipelines:
- ✅ Approach 1: Using OpenAI Embeddings + GPT
- 🐍 Approach 2: Using HuggingFace Embeddings + Local Model (LLaMA or similar)

In [3]:
from pathlib import Path
from langchain.text_splitter import CharacterTextSplitter

# Load HR policy
file_path = r"C:\Users\shruti.y.agarwal\Desktop\Practice\Algorithms\Data\HR_Policy.txt"
text = Path(file_path).read_text()

# Split into chunks
splitter = CharacterTextSplitter(separator="\n", chunk_size=300, chunk_overlap=20)
docs = splitter.create_documents([text])
print("Document chunks:", len(docs))
docs[:2]  # preview


Document chunks: 2


[Document(metadata={}, page_content='Section 1: Leave Policy\nAll full-time employees are entitled to 24 paid leave days per calendar year.\nSick leave requires a medical certificate if more than 2 consecutive days are taken.\nSection 2: Work From Home\nEmployees may request remote work up to 2 days per week, with prior manager approval.'),
 Document(metadata={}, page_content='Section 3: Reimbursement\nEmployees can claim up to â‚¹1,000 per month for internet expenses with valid receipts.')]

## 🔷 Approach 1: OpenAI Embeddings + GPT

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import os

# Set your OpenAI key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Create vector store using OpenAI embeddings
embedding = OpenAIEmbeddings()
db_openai = FAISS.from_documents(docs, embedding)

# Retrieval QA
retriever = db_openai.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-3.5-turbo"), retriever=retriever)

# Ask a question
query = "How many paid leave days are allowed?"
print("Answer:", qa_chain.run(query))


## 🐍 Approach 2: HuggingFace Embeddings + Local Model

In [22]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline

# Load sentence-transformer model for embeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_hf = FAISS.from_documents(docs, embedding)

# Load a local language model for generation
generator = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", device=0)
llm = HuggingFacePipeline(pipeline=generator)

# Create retrieval chain
retriever = db_hf.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Ask the same question
query = "What is the internet reimbursement policy?"
print("Answer:", qa_chain.run(query))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\shruti.y.agarwal\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shruti.y.agarwal\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   3%|2         | 294M/9.95G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/fb/9d/fb9d2cbe2e913461ae463cd35cb5a43ad1c5115ba9e913382f147e2f1ea66f3d/b4515a8e32f81ab4fef65e10891fdb188bc45c4195620d447bc3263e132d9de5?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00002.safetensors%3B+filename%3D%22model-00001-of-00002.safetensors%22%3B&Expires=1750757338&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDc1NzMzOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9mYi85ZC9mYjlkMmNiZTJlOTEzNDYxYWU0NjNjZDM1Y2I1YTQzYWQxYzUxMTViYTllOTEzMzgyZjE0N2UyZjFlYTY2ZjNkL2I0NTE1YThlMzJmODFhYjRmZWY2NWUxMDg5MWZkYjE4OGJjNDVjNDE5NTYyMGQ0NDdiYzMyNjNlMTMyZDlkZTU%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=wDi5JiAcAK7DqyCV3CRR%7Ebv6q822TzT%7EQw94o3T3gpf6ZLFRSgPWwfE--%7Ec5AMMcNSyse0AKncH0A3BjGMCIvwdrEk5vRs--42YixcuOcmG-B0J28a2eU85LVMEtn-S4wDtmYwbiT8xxDfheRgmWwan6KO5W-2-h2klzVku9cKlj6YEiovx3-NOaPxBXlv02zLg%7Es6D4RxM6lAr%7EXThxgXomDvYuDUTsxtx

model-00001-of-00002.safetensors:   3%|2         | 294M/9.95G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/fb/9d/fb9d2cbe2e913461ae463cd35cb5a43ad1c5115ba9e913382f147e2f1ea66f3d/b4515a8e32f81ab4fef65e10891fdb188bc45c4195620d447bc3263e132d9de5?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00002.safetensors%3B+filename%3D%22model-00001-of-00002.safetensors%22%3B&Expires=1750757338&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDc1NzMzOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9mYi85ZC9mYjlkMmNiZTJlOTEzNDYxYWU0NjNjZDM1Y2I1YTQzYWQxYzUxMTViYTllOTEzMzgyZjE0N2UyZjFlYTY2ZjNkL2I0NTE1YThlMzJmODFhYjRmZWY2NWUxMDg5MWZkYjE4OGJjNDVjNDE5NTYyMGQ0NDdiYzMyNjNlMTMyZDlkZTU%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=wDi5JiAcAK7DqyCV3CRR%7Ebv6q822TzT%7EQw94o3T3gpf6ZLFRSgPWwfE--%7Ec5AMMcNSyse0AKncH0A3BjGMCIvwdrEk5vRs--42YixcuOcmG-B0J28a2eU85LVMEtn-S4wDtmYwbiT8xxDfheRgmWwan6KO5W-2-h2klzVku9cKlj6YEiovx3-NOaPxBXlv02zLg%7Es6D4RxM6lAr%7EXThxgXomDvYuDUTsxtx

ValueError: Could not load model tiiuae/falcon-7b-instruct with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.falcon.modeling_falcon.FalconForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "C:\Users\shruti.y.agarwal\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\response.py", line 737, in _error_catcher
    yield
  File "C:\Users\shruti.y.agarwal\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\response.py", line 852, in _raw_read
    data = self._fp_read(amt, read1=read1) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\shruti.y.agarwal\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\response.py", line 835, in _fp_read
    return self._fp.read(a

### 🧩 Note: Required Installations
```
pip install langchain openai faiss-cpu transformers sentence-transformers
```

In [23]:
from transformers import AutoTokenizer

text = "Generative AI is transforming the world."
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer.tokenize(text)
print(tokens)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\shruti.y.agarwal\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shruti.y.agarwal\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

['genera', '##tive', 'ai', 'is', 'transforming', 'the', 'world', '.']


In [24]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

sentences = ["GenAI is powerful.", "Artificial Intelligence is changing the world."]
embeddings = model.encode(sentences)
print(embeddings.shape)

(2, 384)


In [25]:
embeddings

array([[ 2.77056266e-03,  6.38142824e-02, -5.65668754e-02,
         1.18399290e-02, -2.81380191e-02, -2.96664275e-02,
         7.73810893e-02,  2.97305863e-02, -2.82823667e-02,
        -5.14059849e-02,  9.31461528e-03, -1.67999156e-02,
        -4.96530160e-03,  6.25537187e-02,  6.91059977e-02,
         2.53060907e-02,  1.04664244e-01,  3.54761668e-02,
        -4.92834896e-02, -1.07124904e-02,  1.30795629e-03,
         6.20465577e-02,  4.21305187e-02, -2.52282713e-02,
         5.37727997e-02, -2.87194606e-02, -4.10087686e-03,
         1.80802010e-02,  8.18390027e-02, -4.14696299e-02,
        -3.30323051e-03,  1.39598474e-02, -6.36558831e-02,
         2.83321198e-02, -1.08463906e-01,  9.72971171e-02,
        -2.25607809e-02,  8.83382857e-02,  4.58770245e-02,
        -4.43473719e-02,  3.00353058e-02, -2.75530685e-02,
         2.52154637e-02, -3.77236791e-02,  6.16870075e-02,
        -8.33367333e-02, -6.26540557e-02, -2.87474860e-02,
         1.84783502e-03, -3.08106858e-02, -8.69067088e-0